<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Logo_UTFSM.png" width="200" alt="utfsm-logo" align="left"/>

# MAT281
### Aplicaciones de la Matemática en la Ingeniería

## Módulo 04
## Clase 06: Proyectos de Machine Learning

## Objetivos

* Resumir lo que aprendido en el módulo.
* Conocer el _workflow_ de un proyecto de _machine learning_.

## Contenidos
* [Estimadores](#estimator)
* [Pre-Procesamiento](#preprocessing)
* [Pipelines](#pipelines)
* [Evaluación de Modelos](#model_evaluation)
* [Búsqueda de Hiper-Parámetros](#hyperparameter_search)

<a id='estimator'></a>

## Estimadores

Ya sabemos que `scikit-learn` nos provee de múltiples algoritmos y modelos de Machine Learning, que oficialmente son llamados **estimadores** (_estimators_). Cada _estimator_ puede ser ajustado (o coloquialmente, _fiteado_) utilizando los datos adecuados.

Por ejemplo, para motivar, la __Regresión Ridge__ es un tipo de regresión que agrega un parámetro de regularización, en particular, busca minimizar la suma de residuos pero penalizada, es decir:

$$
\min_\beta \vert \vert y - X \beta  \vert \vert_2^2 + \alpha \vert \vert \beta \vert \vert_2^2
$$

El hiper-parámetro $\alpha > 0$ es usualmente conocido como parámetro penalización ridge. En realidad, en la literatura estadística se denota con $lambda$, pero como en `python` el nombre lambda está reservado para las funciones anónimas, `scikit-learn` optó por utilizar otra letra griega. La regresión ridge es una alternativa popularpara sobrellevar el problema de colinealidad.

En `scikit-learn.linear_models` se encuentra el estimador `Ridge`.

In [21]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

from sklearn.datasets import load_boston

In [22]:
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [10]:
rr_est = Ridge(alpha=0.1)

Típicamente el método `fit` acepta dos inputs:

* La matriz de diseño `X`, arreglo bidimensional que típicamente es `(n_samples, n_features)`.
* Los valores _target_ `y`.
    - En tareas de regresión corresponden a números reales.
    - En tareas de clasificación corresopnden a enteros (u otro conjunto de elementos discreto).
    - Para aprendizaje no-supervisado este input no es necesario.

In [11]:
rr_est.fit(X, y)

Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [12]:
rr_est.coef_

array([-1.07473720e-01,  4.65716366e-02,  1.59989982e-02,  2.67001859e+00,
       -1.66846452e+01,  3.81823322e+00, -2.69060598e-04, -1.45962557e+00,
        3.03515266e-01, -1.24205910e-02, -9.40758541e-01,  9.36807461e-03,
       -5.25966203e-01])

In [13]:
rr_est.intercept_

35.69365371165901

El método `predict` necesita un arreglo bidimensional como input. Para ejemplificar podemos utilizar la misma _data_ de entrenamiento.

In [15]:
rr_est.predict(X)[:10]

array([30.04164633, 24.99087654, 30.56235738, 28.65418856, 27.98110937,
       25.28351105, 22.99401212, 19.49937732, 11.46728387, 18.90419332])

En un flujo estándar ajustaríamos con los datos de entrenamiento, predeciríamos datos de test y luego calculamos alguna métrica, por ejemplo, para un caso de regresión, el error cuadrático medio.

In [23]:
rr_est.fit(X_train, y_train)

Ridge(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [32]:
y_pred = rr_est.predict(X_test)

In [35]:
from sklearn.metrics import mean_squared_error

In [36]:
mean_squared_error(y_pred, y_test)

22.14223297423886

<a id='preprocessing'></a>

## Pre-Procesamiento

En el flujo de trabajo típico de un proyecto de machine learning es usual procesar y transformar los datos. En `scikit-learn` el pre-procesamiento y transformación siguen la misma API que los objetos _estimators_, pero que se denotan como _transformers_. Sin embargo, estos no poseen un método `predict` pero si uno de transformación, `transform`.

Motivaremos con la típica estandarización.

In [16]:
from sklearn.preprocessing import StandardScaler

In [18]:
# StandardScaler?

Usualmente se ajusta y transformar los mismos datos, por lo que se aplican los métodos concatenados.

In [19]:
StandardScaler().fit(X).transform(X)

array([[-0.41978194,  0.28482986, -1.2879095 , ..., -1.45900038,
         0.44105193, -1.0755623 ],
       [-0.41733926, -0.48772236, -0.59338101, ..., -0.30309415,
         0.44105193, -0.49243937],
       [-0.41734159, -0.48772236, -0.59338101, ..., -0.30309415,
         0.39642699, -1.2087274 ],
       ...,
       [-0.41344658, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.98304761],
       [-0.40776407, -0.48772236,  0.11573841, ...,  1.17646583,
         0.4032249 , -0.86530163],
       [-0.41500016, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.66905833]])

Sin embargo, muchos de estos objetos (si es que no es la totalidad de ellos), poseen el método `fit_transform`.

In [20]:
StandardScaler().fit_transform(X)

array([[-0.41978194,  0.28482986, -1.2879095 , ..., -1.45900038,
         0.44105193, -1.0755623 ],
       [-0.41733926, -0.48772236, -0.59338101, ..., -0.30309415,
         0.44105193, -0.49243937],
       [-0.41734159, -0.48772236, -0.59338101, ..., -0.30309415,
         0.39642699, -1.2087274 ],
       ...,
       [-0.41344658, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.98304761],
       [-0.40776407, -0.48772236,  0.11573841, ...,  1.17646583,
         0.4032249 , -0.86530163],
       [-0.41500016, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.66905833]])

<a id='pipelines'></a>

## Pipelines

`Scikit-learn` nos permite combinar _transformers_ y _estimators_ uniéndolos a través de "tuberías", objeto denotado como _pipeline_. Nuevamente, la API es consistente con un _estimator_, tanto como para ajustar como para predecir.

In [26]:
from sklearn.pipeline import make_pipeline

In [27]:
pipe = make_pipeline(
    StandardScaler(),
    Ridge(alpha=0.1)
)

In [28]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('ridge',
                 Ridge(alpha=0.1, copy_X=True, fit_intercept=True,
                       max_iter=None, normalize=False, random_state=None,
                       solver='auto', tol=0.001))],
         verbose=False)

In [30]:
pipe.predict(X_test)[:10]

array([28.83639192, 36.00279792, 15.09483565, 25.22983181, 18.87788941,
       23.21453831, 17.59519315, 14.30885051, 23.04885263, 20.62241378])

In [37]:
mean_squared_error(pipe.predict(X_test), y_test)

22.100507974094608

<a id='model_evaluation'></a>

## Evaluación de Modelos

Ya sabemos que ajustar un modelo con datos conocidos no implica que se comportará de buena manera con datos nuevos, por lo que tenemos herramientas como _cross validation_ para evaluar los modelos con los datos conocidos.

In [38]:
from sklearn.model_selection import cross_validate

In [39]:
result = cross_validate(rr_est, X_train, y_train)  # defaults to 5-fold CV

In [40]:
result.keys()

dict_keys(['fit_time', 'score_time', 'test_score'])

In [41]:
result["test_score"]

array([0.73386461, 0.64296157, 0.76353404, 0.77445777, 0.66149893])

<a id='hyperparameter_search'></a>

## Búsqueda de Hiper-parámetros

Para el caso de la regeresión ridge, el parámetro de penalización es un hiper-parámetro que necesita ser escogido con algún procedimiento. Aunque no lo creas, `scikit-learn` también provee herramientas para escoger automáticamente este tipo de hiper-parámetros. 

Por ejemplo `GridSearchCV` realiza una búsqueda exhaustiva entre los posibles valores especificados para los hiper-parámetros.

In [57]:
import numpy as np
from sklearn.model_selection import GridSearchCV

In [64]:
param_grid = {"alpha": np.arange(0, 1, 0.1)}

search = GridSearchCV(
    estimator=rr_est,
    param_grid=param_grid
)

search.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Ridge(alpha=0.1, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [65]:
search.best_params_

{'alpha': 0.0}

El objeto `search` ahora es equivalente a un estimator `Ridge` pero con los mejores parámetros encontrados (`alpha` = 0).

In [66]:
search.score(X_test, y_test)

0.6844267283527127